# Overview of PySpark data management

In this notebook, we will illustrate how various data verbs are implemented in `pyspark`.

## `polars` $\approx$ `pyspark`

You will see a lot of similarities between `polars` and `pyspark`

1. Lazy evaluation and column expression,
2. Parallel processing out-of-the-box,
3. Dot-chained queries, and
4. Data verbs related to `SQL` and/or `dplyr`.

## Data verbs in `pyspark`

In this lecture, we will look at how the common data verbs are implemented in `pyspark`.  Luckily, the implementation is similar to `polars`, so it should be a relatively pain-free transition.

### Overview of Basic Data Verbs in `polars` and `pyspark`

Verb/Function | `polars` | `pyspark` |
--------------|----------|-----------|
Column expr.  | `pl.col('name') ...` | `col('name') ...`|
SELECT | `.select(...)` | `.select(...)` |
FILTER | `.filter(...)` | `.where(...)` |
MUTATE | `.with_columns(...)` | `.withColumn(...)` |
GROUPBY | `.group_by(...)` | `.groupBy(...)`|
AGGREGATE | `.agg(...)` | `.agg(...)` |
JOIN | `l_tbl.join(r_tbl,...)` | `l_tbl.join(r_tbl,...)`|
UNION | `pl.concat` or SQL | `t1.union(t2)` | 
STACK COLUMNS | `.unpivot(...)` | `.unpivot(...)`|
UNSTACK COLUMNS | `.pivot(...)` | `.groupBy(...).pivot(...).<aggfunc>(...)`|

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean

spark = SparkSession.builder.appName('Ops').getOrCreate()
heroes = spark.read.csv('./data/heroes_information.csv', inferSchema=True, header=True)

heroes.limit(5).toPandas()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 07:08:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


,ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99


24/11/12 07:08:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Selecting Columns

The first verb, `select` 

* filters the *columns*
* At the core of `SQL` statements

In [136]:
from pyspark.sql.functions import col

(select_query :=
 heroes
 .select(heroes.name,      # Column via dataframe.name
         col('Gender'),    # Column expression (lazy)
         'Weight')         # String
).limit(5).toPandas()      # <-- outside the saved query

,name,Gender,Weight
0,A-Bomb,Male,441
1,Abe Sapien,Male,65
2,Abin Sur,Male,90
3,Abomination,Male,441
4,Abraxas,Male,-99


In [137]:
select_query  # <-- lazy query

DataFrame[name: string, Gender: string, Weight: int]

## Filtering Rows

The next verb, `filter` 

* filters the *rows*
* is related to the `SQL` `WHERE` clause
* `pyspark`: Use the `where` method

In [139]:
col('Gender') == 'Male' # <-- Lazy column expression

Column<'(Gender = Male)'>

In [140]:
(heroes
 .where(col('Gender') == 'Male')
).limit(5).toPandas()

,ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99


## Chaining Data Verbs

* Processing df $\rightarrow$ chaining data verbs
* Accomplished through dot-chains

In [141]:
(heroes
 .where(col('Gender') == 'Male')
 .select('name', 
         'Gender', 
         'Weight')
).limit(5).toPandas()

,name,Gender,Weight
0,A-Bomb,Male,441
1,Abe Sapien,Male,65
2,Abin Sur,Male,90
3,Abomination,Male,441
4,Abraxas,Male,-99


## Constructing New Columns

The third verb, `mutate` 

* Creates new columns
* Changes existing columns
* `pyspark`: Use the `withColumns` method

### Example 3 - Converting Weight to kilograms

In [142]:
(heroes
 .select('name', 
         'Gender', 
         'Weight')
 .withColumn('Weight_kg', col('Weight')/2.2046)
).limit(5).toPandas()

,name,Gender,Weight,Weight_kg
0,A-Bomb,Male,441,200.036288
1,Abe Sapien,Male,65,29.483807
2,Abin Sur,Male,90,40.823732
3,Abomination,Male,441,200.036288
4,Abraxas,Male,-99,-44.906105


## Referencing a new column

 Use the `col` function with the label from `withColumn`

In [144]:
(new_col_result := 
 heroes
 .select('name', 
         'Gender', 
         'Weight')
 .withColumn('Weight_kg', col('Weight')/2.2046)
 .where(col('Weight_kg') < 100)  # <-- one reason we need lazy expressions
).limit(5).toPandas()

,name,Gender,Weight,Weight_kg
0,Abe Sapien,Male,65,29.483807
1,Abin Sur,Male,90,40.823732
2,Abraxas,Male,-99,-44.906105
3,Absorbing Man,Male,122,55.338837
4,Adam Monroe,Male,-99,-44.906105


## Simple and Grouped Aggregation

In [145]:
(pitching :=  
 spark.read.csv('data/baseball/core/Pitching.csv', inferSchema=True, header=True)
).limit(5).toPandas()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NA,1,2,3,3,2,...,NaN,7,NaN,0,146,0,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NA,12,15,30,30,30,...,NaN,7,NaN,0,1291,0,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NA,0,0,1,0,0,...,NaN,2,NaN,0,14,0,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NA,4,16,24,24,22,...,NaN,20,NaN,0,1080,1,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NA,0,1,1,1,1,...,NaN,0,NaN,0,57,0,21,NaN,NaN,NaN


### Simple Aggregation

A **simple aggregation** collapses all rows into one row.

<img src="https://github.com/wsu-stat489/module5_intro_to_pyspark/blob/main/img/simple_aggregation.png?raw=1" width=800>

In [147]:
from pyspark.sql.functions import mean, std, max, min

(pitching
  .agg(mean('ERA').alias('mean_era'),
       std('ERA').alias('sd_era'),
       max('W').alias('max_wins'),
       min('W').alias('min_wins'))
).toPandas()

,mean_era,sd_era,max_wins,min_wins
0,5.11792,5.466654,60,0


### Group and Aggregate

<img src="https://github.com/wsu-stat489/module5_intro_to_pyspark/blob/main/img/group_and_aggregate.png?raw=1" width=800>

In [149]:
from pyspark.sql.functions import when, col

(eras := 
 pitching
 .where((col('yearID') >= 1900) & (col('yearID') < 1940)) 
 .withColumn('era', (when(col('yearID') < 1920, "dead ball") 
                     .otherwise("after dead ball" ) 
                    )
            )
 .groupby('era')
 .agg(mean('R').alias('mean_runs'))
).toPandas()

,era,mean_runs
0,dead ball,49.707696
1,after dead ball,55.289698


### Grouping by more than one category

* `group_by` accepts multiple columns
* Groups all combinations

In [152]:
from pyspark.sql.functions import sum

(pitching
 .select('yearID', 'teamID', 'W')
 .where(col('yearID') >= 1946)
 .groupby('yearID', 'teamID')
 .agg(sum('W').alias('total_wins'))
 .where(col('total_wins') >= 100)
 .sort(col('yearID').asc(), col('total_wins').desc())
).toPandas()

,yearID,teamID,total_wins
0,1946,BOS,104
1,1953,BRO,105
2,1954,CLE,111
3,1954,NYA,103
4,1961,NYA,109
...,...,...,...
65,2018,NYA,100
66,2019,HOU,107
67,2019,LAN,106
68,2019,NYA,103


## Joins in `pyspark`

Performed with `df_left.join(df_right, how=type_str)`

In [153]:
(dept := 
 spark.read.csv("./data/department.csv",  header=True, inferSchema=True)
).toPandas()

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [154]:
(empl := 
 spark.read.csv("./data/employee.csv",  header=True, inferSchema=True)
).toPandas()

,LastName,DeptID
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,NaN


#### Inner join

In [155]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='inner')
).toPandas()

,LastName,DeptID,DeptID,DeptName
0,Rafferty,31,31,Sales
1,Jones,33,33,Engineering
2,Heisenberg,33,33,Engineering
3,Robinson,34,34,Clerical
4,Smith,34,34,Clerical


#### Left join

In [156]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='left')
).toPandas()

,LastName,DeptID,DeptID,DeptName
0,Rafferty,31.0,31.0,Sales
1,Jones,33.0,33.0,Engineering
2,Heisenberg,33.0,33.0,Engineering
3,Robinson,34.0,34.0,Clerical
4,Smith,34.0,34.0,Clerical
5,Williams,NaN,NaN,None


#### Right join

In [157]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='right')
).toPandas()

,LastName,DeptID,DeptID,DeptName
0,Rafferty,31.0,31,Sales
1,Heisenberg,33.0,33,Engineering
2,Jones,33.0,33,Engineering
3,Smith,34.0,34,Clerical
4,Robinson,34.0,34,Clerical
5,None,NaN,35,Marketing


#### Outer join

In [158]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='outer')
).toPandas()

,LastName,DeptID,DeptID,DeptName
0,Williams,NaN,NaN,None
1,Rafferty,31.0,31.0,Sales
2,Jones,33.0,33.0,Engineering
3,Heisenberg,33.0,33.0,Engineering
4,Robinson,34.0,34.0,Clerical
5,Smith,34.0,34.0,Clerical
6,None,NaN,35.0,Marketing


## Joining on multiple keys

Next, we will look at table joins that require matching multiple keys.

### Example -- Total At Bats, Hits, and Runs Allowed in 2010

To illustrate joining on multiple keys, lets

1. Compute the totals for H and R in 2010 for each team from the `Pitching` table.
2. Join on the team name and park.

This is a good example, because team information can change over the years, so we need to match both `teamID` and `yearID`.

#### Step 1. Read and process the pitching table

In [159]:
(pitching := 
 spark.read.csv("./data/baseball/core/Pitching.csv", header=True, inferSchema=True)
).limit(5).toPandas()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NA,1,2,3,3,2,...,NaN,7,NaN,0,146,0,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NA,12,15,30,30,30,...,NaN,7,NaN,0,1291,0,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NA,0,0,1,0,0,...,NaN,2,NaN,0,14,0,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NA,4,16,24,24,22,...,NaN,20,NaN,0,1080,1,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NA,0,1,1,1,1,...,NaN,0,NaN,0,57,0,21,NaN,NaN,NaN


In [160]:
(teams := 
 spark.read.csv("./data/baseball/core/Teams.csv", header=True, inferSchema=True)
).limit(5).toPandas()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NA,BS1,BNA,None,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NA,CH1,CNA,None,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,1871,NA,CL1,CFC,None,8,29,NaN,10,19,...,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
3,1871,NA,FW1,KEK,None,7,19,NaN,7,12,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
4,1871,NA,NY2,NNA,None,5,33,NaN,16,17,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2


In [161]:
(pitching_totals_2010 := 
 pitching
 .select('teamID', 'yearID', 'R', 'H')
 .where(col('yearID') == 2010)
 .groupBy('teamID', 'yearID')
 .agg(sum('R').alias('Total Runs'), 
      sum('H').alias('Total Hits'))
).limit(5).toPandas()

,teamID,yearID,Total Runs,Total Hits
0,MIN,2010,671,1493
1,CHA,2010,704,1471
2,TOR,2010,728,1407
3,FLO,2010,717,1433
4,TBA,2010,649,1347


#### Step 2. Read and process the teams table

In [162]:
(team_name_and_park := 
 teams
 .select('yearID', 'teamID', col('name').alias('Team Name'), 'park')
).limit(5).toPandas()

,yearID,teamID,Team Name,park
0,1871,BS1,Boston Red Stockings,South End Grounds I
1,1871,CH1,Chicago White Stockings,Union Base-Ball Grounds
2,1871,CL1,Cleveland Forest Citys,National Association Grounds
3,1871,FW1,Fort Wayne Kekiongas,Hamilton Field
4,1871,NY2,New York Mutuals,Union Grounds (Brooklyn)


#### Step 3. Perform a left-join.

Since we want to keep all rows in the totals table, and only add the team information when available, we will perform a left join on the totals table.

Notice that the second `on` argument is now a `list` of column expressions, one for each matching rule.

In [163]:
(pitching_totals_2010
 .join(team_name_and_park,
       on = [pitching_totals_2010.yearID == team_name_and_park.yearID,
             pitching_totals_2010.teamID == team_name_and_park.teamID],
       how='left')
).limit(5).toPandas()

,teamID,yearID,Total Runs,Total Hits,yearID,teamID,Team Name,park
0,MIN,2010,671,1493,2010,MIN,Minnesota Twins,Target Field
1,CHA,2010,704,1471,2010,CHA,Chicago White Sox,U.S. Cellular Field
2,TOR,2010,728,1407,2010,TOR,Toronto Blue Jays,Rogers Centre
3,FLO,2010,717,1433,2010,FLO,Florida Marlins,Dolphin Stadium
4,TBA,2010,649,1347,2010,TBA,Tampa Bay Rays,Tropicana Field


## Concatenating Tables with Set-Like Operations in `pyspark`

Now let's look at combining tables with `union`, `intersect`, and `except` in `pyspark`.

In [164]:
(sales_apr := 
 spark.read.csv("./data/auto_sales_apr.csv",  header=True, inferSchema=True)
).toPandas()

,ID,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


In [165]:
(sales_may := 
 spark.read.csv("./data/auto_sales_may.csv",  header=True, inferSchema=True)
).toPandas()

,ID,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


#### UNION and UNION DISTINCT

In [166]:
(combined_sales :=
 sales_apr
 .union(sales_may)
).toPandas()

,ID,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9
4,0,Ann,22,18,15,12
5,1,Bob,20,14,6,24
6,2,Yolanda,19,10,28,17
7,3,Xerxes,11,27,17,9


In [167]:
(sales_apr
 .union(sales_may)
 .distinct()
).toPandas()

,ID,Salesperson,Compact,Sedan,SUV,Truck
0,3,Xerxes,12,23,18,9
1,2,Yolanda,19,8,32,15
2,1,Bob,19,12,17,20
3,0,Ann,22,18,15,12
4,1,Bob,20,14,6,24
5,3,Xerxes,11,27,17,9
6,2,Yolanda,19,10,28,17


#### Including information from the file name

In [168]:
from pyspark.sql.functions import lit

(combined_sales :=
 sales_apr
 .drop('ID')
 .withColumn('Month', lit('Apr'))  # <-- use `lit` to provide a Java literal (similar to pl.lit in polars)
 .union(sales_may
        .drop('ID')
        .withColumn('Month', lit('May'))
       )
).toPandas()


,Salesperson,Compact,Sedan,SUV,Truck,Month
0,Ann,22,18,15,12,Apr
1,Bob,19,12,17,20,Apr
2,Yolanda,19,8,32,15,Apr
3,Xerxes,12,23,18,9,Apr
4,Ann,22,18,15,12,May
5,Bob,20,14,6,24,May
6,Yolanda,19,10,28,17,May
7,Xerxes,11,27,17,9,May


#### INTERSECTION

In [169]:
(sales_apr
 .intersect(sales_may)
).toPandas()

,ID,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12


#### DIFFERENCE

In [170]:
(sales_apr
 .exceptAll(sales_may)
).toPandas()

,ID,Salesperson,Compact,Sedan,SUV,Truck
0,1,Bob,19,12,17,20
1,2,Yolanda,19,8,32,15
2,3,Xerxes,12,23,18,9


## Reshaping tables

#### Stacking columns with `unpivot`

In [171]:
(combined_sales
 .unpivot(ids = ['Salesperson', 'Month'],
          values = ['Compact','Sedan','SUV','Truck'],
          variableColumnName='Type',
          valueColumnName='Sales'
         )
).toPandas()

,Salesperson,Month,Type,Sales
0,Ann,Apr,Compact,22
1,Ann,Apr,Sedan,18
2,Ann,Apr,SUV,15
3,Ann,Apr,Truck,12
4,Bob,Apr,Compact,19
5,Bob,Apr,Sedan,12
6,Bob,Apr,SUV,17
7,Bob,Apr,Truck,20
8,Yolanda,Apr,Compact,19
9,Yolanda,Apr,Sedan,8


#### Unstacking columns with GROUPBY + PIVOT + SUMMARY METHOD

In [172]:
(combined_sales
 .unpivot(ids = ['Salesperson', 'Month'],
          values = ['Compact','Sedan','SUV','Truck'],
          variableColumnName='Type',
          valueColumnName='Sales'
         )
 .groupBy('Salesperson')
 .pivot('Type')
 .sum('Sales')
).toPandas()

,Salesperson,Compact,SUV,Sedan,Truck
0,Yolanda,38,60,18,32
1,Xerxes,23,35,50,18
2,Ann,44,30,36,24
3,Bob,39,23,26,44


## Review of Basic Data Verbs in `polars` and `pyspark`

Verb/Function | `polars` | `pyspark` |
--------------|----------|-----------|
Column expr.  | `pl.col('name') ...` | `col('name') ...`|
SELECT | `.select(...)` | `.select(...)` |
FILTER | `.filter(...)` | `.where(...)` |
MUTATE | `.with_columns(...)` | `.withColumn(...)` |
GROUPBY | `.group_by(...)` | `.groupBy(...)`|
AGGREGATE | `.agg(...)` | `.agg(...)` |
JOIN | `l_tbl.join(r_tbl,...)` | `l_tbl.join(r_tbl,...)`|
UNION | `pl.concat` or SQL | `t1.union(t2)` | 
STACK COLUMNS | `.unpivot(...)` | `.unpivot(...)`|
UNSTACK COLUMNS | `.pivot(...)` | `.groupBy(...).pivot(...).<aggfunc>(...)`|

## <font color="red"> Exercise 4.4 </font>

Determine all the players that have hit more than 40 home runs in a season in the modern era (e.g., since 1946).  The final table should include the players proper name, as well as the team name. 

**Tasks.**

1. Select and filter where possible,
2. Be sure to aggregate across the stints to compute total HR for each player-year,
3. Remove and keys after joining proper names, and
4. Sort the results by year (outer; ascending) and total HR (inner; descending)

**Hint:** You will need join the files listed below.  To get credit for this exercise, use the join `pyspark` join methods presented above.

In [ ]:
# Your code here